In [72]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

In [73]:
# Starting the webdriver
driver = webdriver.Chrome()

In [74]:
url = 'https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1'
driver.get(url)

## Extracting the features from the first record

In [75]:
soup = BeautifulSoup(driver.page_source,'html.parser')

In [76]:
results = soup.find_all('div',{'data-component-type':'s-search-result'})

In [77]:
item = results[0]

In [78]:
atag = item.h2.a

In [79]:
product_name = atag.text.strip()
product_name

'URBAN TRIBE Havana 15.6" inch Laptop Backpack for Men and Women | 27L Office/Work/Casual Bag | Water Repellent | Black'

In [62]:
url = 'https://www.amazon.com'+atag.get('href')
url

'https://www.amazon.com/sspa/click?ie=UTF8&spc=MTo1ODY1NTY2MTYwNTM2MjEwOjE2NzUyMjkwMDA6c3BfYnRmOjIwMTA4NzQ4ODI1Njk4OjowOjo&url=%2FJustify-Backpack-Compartments-Polyester-Waterproof%2Fdp%2FB0BRGCG7N5%2Fref%3Dsr_1_22_sspa%3Fcrid%3D2M096C61O4MLT%26keywords%3Dbags%26qid%3D1675229000%26sprefix%3Dba%252Caps%252C283%26sr%3D8-22-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9idGY%26psc%3D1'

In [19]:
product_price = item.find('span','a-price')

In [23]:
price = item.find('span','a-offscreen').text
price

'₹998'

In [40]:
rating = item.i.text

In [43]:
rating

'3.9 out of 5 stars'

In [41]:
rating_count = item.find('span',{'class':'a-size-base'}).text
rating_count

'683'

## Function to generalize the above pattern to all products

In [45]:
def extract(item):
    # Product name and url
    atag = item.h2.a
    product_name = atag.text.strip()
    url = 'https://www.amazon.com'+atag.get('href')
    
    try:
        # Product Price
        product_price = item.find('span','a-price')
        price = item.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try:
        # Rating and its count
        rating = item.i.text
        rating_count = item.find('span',{'class':'a-size-base'}).text
    except AttributeError:
        return
    
    result = (product_name,url,price,rating,rating_count)
    
    return result

In [46]:
records = []
results = soup.find_all('div',{'data-component-type':'s-search-result'})

for item in results:
    record = extract(item)
    if record:
        records.append(record)

In [51]:
records[0]

('URBAN TRIBE Havana 15.6" inch Laptop Backpack for Men and Women | 27L Office/Work/Casual Bag | Water Repellent | Black',
 'https://www.amazon.com/sspa/click?ie=UTF8&spc=MTo1ODY1NTY2MTYwNTM2MjEwOjE2NzUyMjkwMDA6c3BfYXRmOjIwMTA0MTQwMjYzNjk4OjowOjo&url=%2FUrban-Tribe-Laptop-Backpack-Havana%2Fdp%2FB01LXNNFDF%2Fref%3Dsr_1_1_sspa%3Fcrid%3D2M096C61O4MLT%26keywords%3Dbags%26qid%3D1675229000%26sprefix%3Dba%252Caps%252C283%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1',
 '₹998',
 '3.9 out of 5 stars',
 '683')

## Going to the next page

In [54]:
def get_url(search):
    template = 'https://www.amazon.in/s?k={}&ref=nb_sb_noss_1'
    search = search.replace(' ','+')
    url = template.format(search)
    # adding page query
    url += '&page{}'
    
    return url

## Getting the details of all products

In [67]:
def extract(item):
    # Product name and url
    atag = item.h2.a
    product_name = atag.text.strip()
    url = 'https://www.amazon.com'+atag.get('href')
    
    try:
        # Product Price
        product_price = item.find('span','a-price')
        price = item.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try:
        # Rating and its count
        rating = item.i.text
        rating_count = item.find('span',{'class':'a-size-base'}).text
    except AttributeError:
        return
    
    result = (product_name,url,price,rating,rating_count)
    
    return result

def get_url(search):
    template = 'https://www.amazon.in/s?k={}&ref=nb_sb_noss_1'
    search = search.replace(' ','+')
    url = template.format(search)
    # adding page query
    url += '&page{}'
    
    return url

# Main Function
def main(search):
    # Starting the webdriver
    driver = webdriver.Chrome()
    
    record = []
    url = get_url(search)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source,'html.parser')
        results = soup.find_all('div',{'data-component-type':'s-search-result'})
        
        for item in results:
            record = extract(item)
            if record:
                records.append(record)
                
    driver.close()
    
    
    # saving all details to csv file
    with open('results.csv','w',newline = '',encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Product Name','Prodcut URL','Product Price','Rating','Rating Count'])
        writer.writerows(records)
    
            
        

In [68]:
main('bag')